In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pylab as pl
import matplotlib.animation as animation
# This is how we can import src module from sibling module
import os, sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path: sys.path.append(nb_dir)
#######################
from src import wdata
from netCDF4 import Dataset
import seaborn as sns;
%matplotlib nbagg
from skimage import io, measure, transform

In [1]:
def rgb2gray(rgb):
    """ Only temporary. Need to use inverse of function used to save radar to RGB .pngs"""
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])

In [2]:
def readFolder(basedir):
    """ 
    Read pix2pix output (test_out) and get input, output, target images
    basedir: path to pix2pix validation output folder (i.e. 'test_out' folder)
    """
    from bs4 import BeautifulSoup as bs
    bshtml = None
    with open(basedir+'index.html', 'rb') as f:
        bshtml = bs(f, 'html.parser')
    tds = bshtml.find_all('td')
    inp, out, targ = [], [], []
    for i in range(0,len(tds),4):
        inp.append(tds[i+1].contents[0]['src'])
        out.append(tds[i+2].contents[0]['src'])
        targ.append(tds[i+3].contents[0]['src'])
#     inp = map(lambda x: io.imread(basedir+x, as_grey=True), inp)
#     out = map(lambda x: io.imread(basedir+x, as_grey=True), out)
#     targ = map(lambda x: io.imread(basedir+x, as_grey=True), targ)
#     inp = map(lambda x: plt.imread(basedir+x), inp)
#     out = map(lambda x: plt.imread(basedir+x), out)
#     targ = map(lambda x: plt.imread(basedir+x), targ)
#     inp = map(lambda x: plt.imshow(x, cmap='gray').get_array().data, inp)
#     out = map(lambda x: plt.imshow(x, cmap='gray').get_array().data, out)
#     targ = map(lambda x: plt.imshow(x, cmap='gray').get_array().data, targ)
#     plt.close()
    inp = map(lambda x: rgb2gray(plt.imread(basedir+x)), inp)
    out = map(lambda x: rgb2gray(plt.imread(basedir+x)), out)
    targ = map(lambda x: rgb2gray(plt.imread(basedir+x)), targ)

    return (np.array(inp), np.array(out), np.array(targ))

In [ ]:
savedir = 'c:/Users/Sa/Desktop/pres6/'
basedir = 'c:/Users/Sa/Desktop/weather/'

In [ ]:
def diffL1(x,y):
    """ L1 distance between 2 tensors """
    return np.sum(np.abs(x-y))*1.0/np.prod(x.shape)
# inp, out, targ = readFolder(basedir)
# print len(inp), len(out), len(targ)
# errs = []
# l1s = []
# for x,y in zip(out,targ):
# #     ax, fig = plt.subplots(ncols=2,nrows=1)
# #     fig[0].imshow(x)
# #     fig[1].imshow(y)
# #     plt.show()
#     err = measure.compare_nrmse(x, y)
#     errs += [err]
# #     print err
#     l1s += [diffL1(x,y)]
# print np.min(errs), np.max(errs), np.average(errs)
# print np.min(l1s), np.max(l1s), np.average(l1s)
# his = np.histogram(l1s)
# # plt.hist(his[1],his[0])
# # plt.show()
# plt.scatter(his[1][:-1],his[0])
# plt.show()

In [3]:
# %matplotlib nbagg
def pyramidL2(img1, img2, show=False):
    """ 
    compare how similar two images are using pyrmid downscaling and RMSE 
    return: [(shape, rmse), .... ]
    """
    l2s = []
    for i,j in zip(transform.pyramid_gaussian(img1), transform.pyramid_gaussian(img2))[:6]:
        shape, rmse = i.shape, np.sqrt(measure.compare_mse(i,j))
        l2s.append((shape,rmse))
        if not show: continue
        ax, fig = plt.subplots(ncols=2,nrows=1)
        fig[0].imshow(i,cmap='gray')
        fig[0].set_xticks([])
        fig[0].set_yticks([])
        fig[1].imshow(i,cmap='gray')
        fig[1].set_xticks([])
        fig[1].set_yticks([])
        ax.text(.92, 0.4, 'dim: %s'%str(shape), fontsize=18)
        ax.text(.92, 0.6, 'RMSE: %s'%str(rmse)[:5], fontsize=18)
        ax.subplots_adjust(left=0.1)
#         ax.savefig(savedir+str(shape[0])+'.png', bbox_inches='tight')
        plt.show()
    return l2s
  
# print pyramidL2(out[0], targ[0], True)

In [4]:
def evaluateModel(rootpath):
    """ use RMSE an pyrmaid method to score a pix2pix model """
    _, mout, mtarg = readFolder(rootpath)
    allrmse = [pyramidL2(x,y) for x,y in zip(mout,mtarg)]
    avgByDim = {k[0]: 0.0 for k in allrmse[0]}
    for img in allrmse:
        for dim,rmse in img:
            avgByDim[dim] += rmse
    for k in avgByDim.iterkeys():     
        avgByDim[k] /= len(allrmse)
    return avgByDim

In [ ]:
# evaluate each of our models:
old2 = evaluateModel(basedir+'2old/test_out/')
lowerThresh2 = evaluateModel(basedir+'2lower_threshold/test_out/')
bands2and4 = evaluateModel(basedir+'2and4/test_out/')
band6 = evaluateModel(basedir+'6/test_out/')

In [ ]:
sortkeys = sorted(old2.keys())
width = 0.1 
plt.xticks(np.arange(6), [str(k)[1:-1].replace(', ','x').replace('L','') for k in sortkeys])
for i, (model, desc) in enumerate(zip([old2, lowerThresh2, bands2and4, band6], ['Band 2 (non-normalized)', 'Band 2 (less-strict sparsity threshold)', 'Bands 2 and 4 (stacked)', 'Band 6'])):
#     plt.figure()
    plt.bar(np.arange(6)+width*i, [model[k] for k in sortkeys],width=width, label=desc)
plt.title("Models RMSE by dimension (eval on test data only)")
plt.legend()
plt.show()
